In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                            train=False,
                            transform=transforms.ToTensor())

In [3]:
batch_size = 200
n_iters = 60000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [4]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self):
        super(FeedforwardNeuralNetModel, self).__init__()
        
        self.l1 = nn.Linear(28*28, 800)
        self.l2 = nn.Linear(800, 700)
        self.l3 = nn.Linear(700, 600)
        self.l4 = nn.Linear(600, 500)
        self.l5 = nn.Linear(500, 400)
        self.l6 = nn.Linear(400, 300)
        self.l7 = nn.Linear(300, 200)
        self.l8 = nn.Linear(200, 100)
        self.l9 = nn.Linear(100, 50)
        
        
    def forward(self, x):
        
        model = torch.nn.Sequential(
            self.l1,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l2,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l3,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l4,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l5,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l6,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l7,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l8,
            nn.Dropout(0.2),
            nn.ReLU(),
            self.l9
        )
        return model(x)
        

In [5]:
model = FeedforwardNeuralNetModel()

In [6]:
# Automatically calculates softmax and crossentropy function
criterion = nn.CrossEntropyLoss()

In [7]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(model.parameters)

<bound method Module.parameters of FeedforwardNeuralNetModel(
  (l1): Linear(in_features=784, out_features=800, bias=True)
  (l2): Linear(in_features=800, out_features=700, bias=True)
  (l3): Linear(in_features=700, out_features=600, bias=True)
  (l4): Linear(in_features=600, out_features=500, bias=True)
  (l5): Linear(in_features=500, out_features=400, bias=True)
  (l6): Linear(in_features=400, out_features=300, bias=True)
  (l7): Linear(in_features=300, out_features=200, bias=True)
  (l8): Linear(in_features=200, out_features=100, bias=True)
  (l9): Linear(in_features=100, out_features=50, bias=True)
)>


In [8]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted.cpu() == labels.cpu()).sum()
            
            accuracy = (correct*100) / total 
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.334857225418091. Accuracy: 10
Iteration: 1000. Loss: 2.3040339946746826. Accuracy: 9
Iteration: 1500. Loss: 2.340137243270874. Accuracy: 10
Iteration: 2000. Loss: 2.308499813079834. Accuracy: 9
Iteration: 2500. Loss: 1.8864548206329346. Accuracy: 23
Iteration: 3000. Loss: 1.3415236473083496. Accuracy: 45
Iteration: 3500. Loss: 0.8568657636642456. Accuracy: 67
Iteration: 4000. Loss: 0.5636377930641174. Accuracy: 80
Iteration: 4500. Loss: 0.18930204212665558. Accuracy: 93
Iteration: 5000. Loss: 0.1792113482952118. Accuracy: 94
Iteration: 5500. Loss: 0.1252259612083435. Accuracy: 94
Iteration: 6000. Loss: 0.16597965359687805. Accuracy: 95
Iteration: 6500. Loss: 0.1299264132976532. Accuracy: 96
Iteration: 7000. Loss: 0.1141321212053299. Accuracy: 96
Iteration: 7500. Loss: 0.009596929885447025. Accuracy: 96
Iteration: 8000. Loss: 0.09314536303281784. Accuracy: 96
Iteration: 8500. Loss: 0.03849339857697487. Accuracy: 96
Iteration: 9000. Loss: 0.06294231861829758. Accu

In [9]:
torch.save(model.state_dict(), './model.pth')